In [65]:
"""TRABALHO 3: PARTE 1 - Algoritmo Naïve Bayes.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/18Y-yMmG5njAv0vRM2_hcTdFDuneopATt

# PARTE 1: Algoritmo Naïve Bayes

Nesta primeira parte do Trabalho você irá aplicar o algoritmo de Naïve Bayes na base de dados de risco de crédito discutida em aula. Para isso você deve primeiramente importar as bibliotecas necessárias.
"""

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB

label_encoder_historia = LabelEncoder()
label_encoder_divida = LabelEncoder()
label_encoder_garantia = LabelEncoder()
label_encoder_renda = LabelEncoder()
label_encoder_risco = LabelEncoder()

def carregar_dados():
  dataset_risco_credito = pd.read_csv("dataset_risco_credito.csv");
  # imprima a base de dados para uma primeira avaliação dos dados
  print("Avaliar os dados")
  print(dataset_risco_credito)
  print()
  # De acordo com o slide 17, são as características (todas as colunas menos a última)
  X_risco_credito = dataset_risco_credito.iloc[:, :-1].values
  # Última coluna, de risco.
  y_risco_credito = dataset_risco_credito.iloc[:, -1].values

  return X_risco_credito, y_risco_credito

def aplicar_label_encoder(X_risco_credito, y_risco_credito):
  # vai codificar cada uma das colunas dos atributos para numérico
  X_risco_credito[:, 0] = label_encoder_historia.fit_transform(X_risco_credito[:, 0])
  X_risco_credito[:, 1] = label_encoder_divida.fit_transform(X_risco_credito[:, 1])
  X_risco_credito[:, 2] = label_encoder_garantia.fit_transform(X_risco_credito[:, 2])
  X_risco_credito[:, 3] = label_encoder_renda.fit_transform(X_risco_credito[:, 3])

  # codifica a coluna do risco (classe) para numérico também
  y_risco_credito = label_encoder_risco.fit_transform(y_risco_credito)

  # Apresente o resultado do label enconder
  print("\nResultado dos label encoders:")
  print(X_risco_credito)
  print(y_risco_credito)

  return X_risco_credito, y_risco_credito

def salvar_dados(nome_arquivo, X_risco_credito, y_risco_credito):
  with open(nome_arquivo, 'wb') as f:
      pickle.dump([X_risco_credito, y_risco_credito], f)

def treinar_modelo(X_risco_credito, y_risco_credito):
  naiveb_risco_credito = GaussianNB()
  naiveb_risco_credito.fit(X_risco_credito, y_risco_credito)
  return naiveb_risco_credito

def montar_dados_codificados(atributo1, atributo2, atributo3, atributo4):
  historia_codificado = label_encoder_historia.transform([atributo1])
  divida_codificado = label_encoder_divida.transform([atributo2])
  garantia_codificado = label_encoder_garantia.transform([atributo3])
  renda_codificado = label_encoder_renda.transform([atributo4])

  return np.array([historia_codificado[0], divida_codificado[0], garantia_codificado[0], renda_codificado[0]])

def prever_risco(naiveb_risco_credito, dados_codificados):
  previsao = naiveb_risco_credito.predict([dados_codificados])
  return previsao

def decodificar_previsao(label_encoder_risco, previsao):
  classe_prevista = label_encoder_risco.inverse_transform(previsao)[0]
  return classe_prevista


'''
# 1 - Pré-Processamento dos dados

a) DIVISÃO DA BASE DE DADOS

Separe a base de dados dataset_risco_credito em:
 - variável x, com nome: X_risco_credito
 - classe y, com nome: y_risco_credito

 DICA: você pode utilizar .iloc para selecionar as colunas da matriz e .values para converter para o numpy array.

'''
X_risco_credito, y_risco_credito = carregar_dados()

'''
b) APLICAR LABEL ENCODER

Perceba que seus dados possuem atributos categóricos (string). Porém, para aplicar esses dados em um algoritmo de aprendizado você precisa transformá-lo em atributo numérico.

Como você pode resolver isso?

DICA: Veja o que é e como aplicar o Label Enconder em: https://youtu.be/nLKEkBAbpQo
'''
X_risco_credito, y_risco_credito = aplicar_label_encoder(X_risco_credito, y_risco_credito)


'''c) SALVAR O ARQUIVO PRÉ-PROCESSADO'''
salvar_dados("risco_credito.pkl", X_risco_credito, y_risco_credito)


'''# 2 - Algoritmo Naïve Bayes'''
'''
a) TREINAR O ALGORITMO

Para treinar o algoritmo, você deve gerar a tabela de probabilidades. Para isso, você pode utilizar **.fit** para gerar a tabela.

DICA: O 1º parametro são os atributos/características (x) e o 2º parametro é a classe (y).

OBS: Não se preocupe, o algoritmo faz a correção laplaciana automaticamente :) .
'''
naiveb_risco_credito = treinar_modelo(X_risco_credito, y_risco_credito)

'''
b) FAZER A PREVISÃO

Utilize **.predict** para fazer a previsão realizada no exemplo em sala.

i) história boa, dívida alta, garantia nenhuma, renda > 35

ii) história ruim, dívida alta, garantia adequada, renda < 15

Verifique nos slides se seu resultado está correto!

# utilize o atributo .classes_ para mostrar as classes utilizadas pelo algoritmo

# utilize .class_count_ para contar quantos registros tem em cada classe
'''

# i)
dados_codificados_i = montar_dados_codificados("boa", "alta", "nenhuma", "acima_35")
previsao_i = prever_risco(naiveb_risco_credito, dados_codificados_i)

# decodificar a previsão (pois o método predict retorna a previsão numérica,
# é necessário decodificar para ter o resultado alfanumérico)
classe_prevista_i = decodificar_previsao(label_encoder_risco, previsao_i)

print("Previsão para i:", classe_prevista_i)


# ii)
dados_codificados_ii = montar_dados_codificados("ruim", "alta", "adequada", "0_15")
previsao_ii = prever_risco(naiveb_risco_credito, dados_codificados_ii)

# decodificar a previsão (pois o método predict retorna a previsão numérica,
# é necessário decodificar para ter o resultado alfanumérico)
classe_prevista_ii = decodificar_previsao(label_encoder_risco, previsao_ii)

print("Previsão para ii:", classe_prevista_ii)




Avaliar os dados
        historia divida garantias     renda     risco
0           ruim   alta   nenhuma      0_15      alto
1   desconhecida   alta   nenhuma     15_35      alto
2   desconhecida  baixa   nenhuma     15_35  moderado
3   desconhecida  baixa   nenhuma  acima_35      alto
4   desconhecida  baixa   nenhuma  acima_35     baixo
5   desconhecida  baixa  adequada  acima_35     baixo
6           ruim  baixa   nenhuma      0_15      alto
7           ruim  baixa  adequada  acima_35  moderado
8            boa  baixa   nenhuma  acima_35     baixo
9            boa   alta  adequada  acima_35     baixo
10           boa   alta   nenhuma      0_15      alto
11           boa   alta   nenhuma     15_35  moderado
12           boa   alta   nenhuma  acima_35     baixo
13          ruim   alta   nenhuma     15_35      alto


Resultado dos label encoders:
[[2 0 1 0]
 [1 0 1 1]
 [1 1 1 1]
 [1 1 1 2]
 [1 1 1 2]
 [1 1 0 2]
 [2 1 1 0]
 [2 1 0 2]
 [0 1 1 2]
 [0 0 0 2]
 [0 0 1 0]
 [0 0 1 1]
 [0 0 1 2